In [1]:
import sklearn
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def get_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return {
        'mean_squared_error': mse,
        'root_mean_squared_error': rmse,
        'mean_absolute_error': mae,
        'r_squared': r2
    }


In [4]:
state = 356
cv_obj = KFold(n_splits=10, shuffle=True, random_state=state)

# Tree Grid Search

In [5]:
Salary = pl.read_csv('./Data/Salary_Bball.csv')
SalaryTrain = Salary.filter(pl.col('Training-Validation') == 'Training')
SalaryTest = Salary.filter(pl.col('Training-Validation') == 'Validation')
Salary

playerID,yearID,salary,stint,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old,W,L,G_right,GS,CG,SHO,SV,IPouts,H_right,ER,HR_right,BB_right,SO_right,BAOpp,ERA,IBB_right,WP,HBP_right,BK,BFP,GF,R_right,SH_right,SF_right,GIDP_right,GS_right,InnOuts,PO,A,E,DP,PB,WP_right,SB_right,CS_right,ZR,Training-Validation
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""pillbr01""",2012,480000,1,48,0,105,10,22,3,0,4,11,1,0,6,19,1,2,1,0,5,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
"""hamptmi01""",2009,2000000,1,21,0,37,6,12,1,0,1,7,0,0,2,8,0,0,1,1,0,0,7,10,21,21,0,0,0,336,128,66,13,46,74,0.298,5.3,6,1,2,0,494,0,71,11,5,13,21,336,9,17,2,1,0,0,0,0,0,"""Training"""
"""mathisc01""",2007,380000,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
"""hansoto01""",2013,3725000,1,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,3,15,13,0,0,0,219,83,44,10,30,56,0.286,5.42,3,6,3,0,327,1,47,0,4,3,13,219,7,7,1,0,0,0,0,0,0,"""Training"""
"""wrighja01""",2006,800000,1,35,0,47,4,12,2,0,0,1,0,0,0,11,0,0,5,0,1,0,6,10,34,21,0,0,0,468,167,90,16,64,79,0.282,5.19,4,6,10,0,676,2,95,5,4,24,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""espinda01""",2015,1800000,1,118,0,367,59,88,21,1,13,37,5,2,33,106,5,6,3,3,6,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
"""pradoma01""",2016,11000000,1,153,0,600,70,183,37,3,8,75,2,2,49,69,4,4,0,5,24,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Validation"""
"""derosma01""",2013,750000,1,88,0,204,23,48,12,1,7,36,0,0,28,49,1,1,0,3,8,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""


In [6]:
Salary_Stats = SalaryTrain.select(pl.col('G'), pl.col('ZR'))
Salary = SalaryTrain.select(pl.col('salary'))
Salary_Stats_Test = SalaryTest.select(pl.col('G'), pl.col('ZR'))
Salary_Test = SalaryTest.select(pl.col('salary'))

In [7]:
param_grid = {'criterion': ['squared_error', 'friedman_mse'],
              'max_depth': [10, 20, None],
              'min_samples_split': [2, 10],
              'min_samples_leaf': [1, 5],
            }

In [8]:
tree_grid_search = GridSearchCV(DecisionTreeRegressor(), param_grid, verbose=3, scoring='neg_mean_squared_error', cv = cv_obj)

tree_grid_search.fit(Salary_Stats, Salary)

Fitting 10 folds for each of 24 candidates, totalling 240 fits
[CV 1/10] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=-21177605610575.488 total time=   0.0s
[CV 2/10] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=-19285461720898.344 total time=   0.0s
[CV 3/10] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=-17994983384439.340 total time=   0.0s
[CV 4/10] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=-19398992551021.047 total time=   0.0s
[CV 5/10] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=-17554193700521.236 total time=   0.0s
[CV 6/10] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=2;, score=-21319819129583.422 total time=   0.0s
[CV 7/10] END criterion=squared_error, max_depth=10, min_samples_leaf=1, min_samples_split=2;, sc

GridSearchCV(cv=KFold(n_splits=10, random_state=356, shuffle=True),
             estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['squared_error', 'friedman_mse'],
                         'max_depth': [10, 20, None],
                         'min_samples_leaf': [1, 5],
                         'min_samples_split': [2, 10]},
             scoring='neg_mean_squared_error', verbose=3)

In [9]:
tree_grid_search.best_score_

np.float64(-19625590129809.508)

In [10]:
tree_grid_search.best_params_

{'criterion': 'squared_error',
 'max_depth': 10,
 'min_samples_leaf': 5,
 'min_samples_split': 2}

In [11]:
tree_grid_search.best_estimator_

DecisionTreeRegressor(max_depth=10, min_samples_leaf=5)

In [12]:
Salary_Pred = tree_grid_search.predict(Salary_Stats)

Salary_Pred

array([2210760.8989899 , 3500929.81481481, 2076944.47959184, ...,
       2800565.625     , 3357841.08      , 2516874.28571429])

In [13]:
get_metrics(Salary, Salary_Pred)

{'mean_squared_error': np.float64(18902150587093.652),
 'root_mean_squared_error': np.float64(4347660.357835425),
 'mean_absolute_error': np.float64(3084256.5982865333),
 'r_squared': 0.10404366876133031}

# Forest Grid Search

In [14]:
rf_param_grid = {'criterion': ['squared_error', 'friedman_mse'],
                 'n_estimators': [10, 100],
                  'max_depth': [None, 1,2,3,4,5],
                   'min_samples_leaf' : [1, 5, 10],
                   'bootstrap': [True, False],
                    'oob_score': [True, False] 
              }

In [15]:
rf_grid_search = GridSearchCV(RandomForestRegressor(), rf_param_grid, verbose=3, scoring='neg_mean_squared_error', cv = cv_obj)

rf_grid_search.fit(Salary_Stats, Salary)

Fitting 10 folds for each of 288 candidates, totalling 2880 fits
[CV 1/10] END bootstrap=True, criterion=squared_error, max_depth=None, min_samples_leaf=1, n_estimators=10, oob_score=True;, score=-21315936573506.125 total time=   0.0s


C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\ensemble\_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\ensemble\_forest.py:615: UserWarning: Some inputs do not have OOB scores. This pr

[CV 2/10] END bootstrap=True, criterion=squared_error, max_depth=None, min_samples_leaf=1, n_estimators=10, oob_score=True;, score=-19329151565160.242 total time=   0.0s
[CV 3/10] END bootstrap=True, criterion=squared_error, max_depth=None, min_samples_leaf=1, n_estimators=10, oob_score=True;, score=-17993914532143.188 total time=   0.1s


C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\ensemble\_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\ensemble\_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sa

[CV 4/10] END bootstrap=True, criterion=squared_error, max_depth=None, min_samples_leaf=1, n_estimators=10, oob_score=True;, score=-19384035454632.367 total time=   0.1s
[CV 5/10] END bootstrap=True, criterion=squared_error, max_depth=None, min_samples_leaf=1, n_estimators=10, oob_score=True;, score=-17517880409224.004 total time=   0.0s


...

[CV 9/10] END bootstrap=False, criterion=friedman_mse, max_depth=5, min_samples_leaf=10, n_estimators=100, oob_score=False;, score=-19340637684919.961 total time=   0.8s


C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV 10/10] END bootstrap=False, criterion=friedman_mse, max_depth=5, min_samples_leaf=10, n_estimators=100, oob_score=False;, score=-19503321454314.055 total time=   0.8s


C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
720 fits failed out of a total of 2880.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
720 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jb5983on\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jb5983on\Ap

GridSearchCV(cv=KFold(n_splits=10, random_state=356, shuffle=True),
             estimator=RandomForestRegressor(),
             param_grid={'bootstrap': [True, False],
                         'criterion': ['squared_error', 'friedman_mse'],
                         'max_depth': [None, 1, 2, 3, 4, 5],
                         'min_samples_leaf': [1, 5, 10],
                         'n_estimators': [10, 100],
                         'oob_score': [True, False]},
             scoring='neg_mean_squared_error', verbose=3)

In [16]:
rf_grid_search.best_score_

np.float64(-19347971994730.36)

In [17]:
rf_grid_search.best_params_

{'bootstrap': True,
 'criterion': 'friedman_mse',
 'max_depth': 4,
 'min_samples_leaf': 1,
 'n_estimators': 100,
 'oob_score': True}

In [18]:
rf_grid_search.best_estimator_

RandomForestRegressor(criterion='friedman_mse', max_depth=4, oob_score=True)

In [19]:
Salary_Forest_Pred = rf_grid_search.predict(Salary_Stats)

Salary_Forest_Pred

array([2112635.51558817, 2974317.45097508, 2121608.11705291, ...,
       2776145.121519  , 3616574.08736033, 2832214.29882626])

In [20]:
get_metrics(Salary, Salary_Forest_Pred)

{'mean_squared_error': np.float64(19109984429621.86),
 'root_mean_squared_error': np.float64(4371496.817981441),
 'mean_absolute_error': np.float64(3110612.281265535),
 'r_squared': 0.09419240627133885}

In [21]:
rf_grid_search.best_score_ > tree_grid_search.best_score_

np.True_

In [22]:
Winning_Model = rf_grid_search.predict(Salary_Stats_Test)

In [23]:
get_metrics(Salary_Test, Winning_Model)

{'mean_squared_error': np.float64(32895867351149.332),
 'root_mean_squared_error': np.float64(5735491.9014108395),
 'mean_absolute_error': np.float64(3806219.935581305),
 'r_squared': 0.054400295023073975}